## 数据导入

首先是导入 Pandas 软件包。

In [1]:
import pandas as pd

下面读入从 Netlogo 的行为空间生成的 csv 文件。注意这个文件的开头几行分别是：

![](https://tva1.sinaimg.cn/large/006tNbRwly1gblq05ddawj30bm06u3zl.jpg)

所以，我们需要使用 `skiprows` 来跳过前面这几行，然后因为剩下的内容里面，第一列是标题，所以这里把它设定成为 `index_col` 。

In [2]:
df = pd.read_csv("lesson8-gini-spreadsheet.csv", skiprows=6, index_col=0)

来看看此时的数据框：

In [3]:
df

,1,2,3,4,5,6,7,8,9,10,...,1991,1992,1993,1994,1995,1996,1997,1998,1999,2000
[run number],,,,,,,,,,,,,,,,,,,,,
total-money,10000,10000,10000,10000,10000,10000,10000,10000,10000,10000,...,100000,100000,100000,100000,100000,100000,100000,100000,100000,100000
num-agents,100,100,100,100,100,100,100,100,100,100,...,1000,1000,1000,1000,1000,1000,1000,1000,1000,1000
saving?,false,false,false,false,false,false,false,false,false,false,...,true,true,true,true,true,true,true,true,true,true
[steps],1000,1000,1000,1000,1000,1000,1000,1000,1000,1000,...,1000,1000,1000,1000,1000,1000,1000,1000,1000,1000
[initial & final values],compute-gini,compute-gini,compute-gini,compute-gini,compute-gini,compute-gini,compute-gini,compute-gini,compute-gini,compute-gini,...,compute-gini,compute-gini,compute-gini,compute-gini,compute-gini,compute-gini,compute-gini,compute-gini,compute-gini,compute-gini
NaN,0.42062473965752134,0.5006348336058749,0.49393353338078494,0.5073621024743512,0.4853796586466097,0.45746503189312104,0.4573222810974193,0.5239413840065117,0.47050704178887803,0.4842621233558364,...,0.6995618052497102,0.7754577175033197,0.7856686680925841,0.7339137273175398,0.7104869820317155,0.7560097962529742,0.779969921453448,0.743097027090142,0.7512450510464614,0.774020884778738


显然，我们更喜欢把整个数据框转 90 度的样子。在 Pandas 里面，转置 90 度，需要使用 `.T`。

转置之后，咱们看看前几行。

In [4]:
df = df.T; df.head()

[run number],total-money,num-agents,saving?,[steps],[initial & final values],NaN
1,10000,100,false,1000,compute-gini,0.42062473965752134
2,10000,100,false,1000,compute-gini,0.5006348336058749
3,10000,100,false,1000,compute-gini,0.49393353338078494
4,10000,100,false,1000,compute-gini,0.5073621024743512
5,10000,100,false,1000,compute-gini,0.4853796586466097


这里面，有没有不需要的列？

当然有。

例如 `run number`，这个只是索引而已。

例如 `steps` ，只是提示我们运行了多少步。这里是每一组实验，都运行了 1000 步，所以没有保留价值。

还有 `initial & final values` 这一项，下面对应的，不是真正的数值，而是使用的报告函数名称。这个也没有必要保留。

所以，咱们实际上需要保留的，是另外那几列。

在 Pandas 里面，可以利用 `iloc` 来指定需要保留的行列。这里咱们保留所有行，但是扔掉刚才提及的列。

In [5]:
df = df.iloc[:, [0, 1, 2, 5]]
df.columns = ["total_money", "num_agents", "saving", "gini"]
df.head()

,total_money,num_agents,saving,gini
1,10000,100,false,0.42062473965752134
2,10000,100,false,0.5006348336058749
3,10000,100,false,0.49393353338078494
4,10000,100,false,0.5073621024743512
5,10000,100,false,0.4853796586466097


注意这里除了列选择以外，咱们还用 `columns` 修改了列名称。

现在看起来，感觉好多了。

## 聚合操作


这里每一组的实验，我们实际上做了 10 次。这里主要是为了对抗随机性带来的偏差。所以平均值更能说明问题。

我们尝试一下，利用 Pandas 里面的 `groupby()` 分组求平均。

In [6]:
df = df.groupby(["total_money", "num_agents", "saving"]).mean().reset_index(); df.head()

DataError: No numeric types to aggregate

看起来，效果很不乐观啊。

为什么呢？

难道是语法错误？

注意这里的报错信息，说的是”没有数值类型来进行聚合操作“。

问题来了，明明有啊。难道数据框里面的那些数，有问题吗？

这时候，就需要你仔细了。

你得验证一下。人看上去是数值，机器看上去呢？

这里我们尝试用一下 Pandas 里面的 `dtypes` 查看一下每一列的类型

In [7]:
df.dtypes

total_money    object
num_agents     object
saving         object
gini           object
dtype: object

这里你会看到，都是 `object` 类型。那么什么是`object` 类型呢？其实就是字符串。

也就是说，你虽然看到了一堆数字和布尔值，但是在机器看来，它们都只是字符串而已。

怎么办？

我们转换它们。

转换的方式，可是因不同列而异。

例如说，对于数值型的，我们直接转换成整型（`int`）或者浮点型（`float`）就可以。需要调用的函数是 `astype()` 。

In [8]:
df["total_money"] = df["total_money"].astype(float)
df["num_agents"] = df["num_agents"].astype(int)
df["gini"] = df["gini"].astype(float)

但是，对于布尔型，就不是那么容易了。

因为在 netlogo 里面，布尔型用 `true` 或者 `false` 来表示。

而在 Python 里面，我们用 `True` 或者 `False` 。所以直接转换，会发生识别上的错误。

怎么办呢？

我们直接指定一个匿名函数，把字符串里面的 `true`，转换成布尔型的`True` 。



In [9]:
df.saving = df.saving.apply(lambda x: True if x=="true" else False)

此时，咱们再来看看数据框。

In [10]:
df.head()

,total_money,num_agents,saving,gini
1,10000.0,100,False,0.420625
2,10000.0,100,False,0.500635
3,10000.0,100,False,0.493934
4,10000.0,100,False,0.507362
5,10000.0,100,False,0.485380


除了 `false` 的首字母大写了，好像也没有什么变化啊！

别忙，我们尝试一下，看类型。

In [11]:
df.dtypes

total_money    float64
num_agents       int64
saving            bool
gini           float64
dtype: object

看，这次没问题了。

好了，让我们再度尝试一下，进行聚合，也就是 10 次随机试验的结果求均值。

In [12]:
df = df.groupby(["total_money", "num_agents", "saving"]).mean().reset_index(); df.head()

,total_money,num_agents,saving,gini
0,10000.0,100,False,0.480143
1,10000.0,100,True,0.606903
2,10000.0,200,False,0.495969
3,10000.0,200,True,0.662886
4,10000.0,300,False,0.493032


目标达成！

验证一下，看求均值聚合操作以后，还剩下多少行？

In [13]:
len(df)

200

这就对了！

## 可视化

这里，我们采用 altair 进行可视化操作。这是 Python 数据可视化领域的后起之秀。

首先读入软件包。

In [14]:
import altair as alt

然后，我们先来看看，锁定其中一个变量。例如 `num_agents` 固定为 100 的时候，其他变量的变化趋势。

In [15]:
temp_df = df[df["num_agents"]==100]

In [16]:
temp_df

,total_money,num_agents,saving,gini
0,10000.0,100,False,0.480143
1,10000.0,100,True,0.606903
20,20000.0,100,False,0.488479
21,20000.0,100,True,0.655820
40,30000.0,100,False,0.473415
41,30000.0,100,True,0.631622
60,40000.0,100,False,0.477590
61,40000.0,100,True,0.623049
80,50000.0,100,False,0.473747
81,50000.0,100,True,0.602335


In [17]:
len(temp_df)

20

验证了一下长度，没问题。确实是 20 组数据。

然后我们就可以运用 Altair 的语句，来做出可视化。

In [37]:
alt.Chart(temp_df).mark_point().encode(
    alt.X("total_money:Q"),
    alt.Y("gini:Q"),
    alt.Color("saving:N")
).properties(width=600)

alt.Chart(...)

这里解释一下，其中：

- `alt.Chart` 里面指定要进行可视化的数据框，这里是 `temp_df` ；
- 你需要指定绘图的类型。因为我们打算在不同的总财富数量上，描摹基尼系数的变化，所以这里可以采用散点图 `mark_point()`；
- `encode` 用于指定 X 轴，Y 轴和其他要素。这里是把需要画的内容映射到它们；
- X 轴上，用的是总财富 `total_money` ，`:Q` 用于指定变量类型。这里我们将其视为量化数据。 Y 轴与之类似；
- `alt.Color()` 用于将 `saving` 变量根据真值类型映射到不同的颜色上。
- `properties` 用于指定图的一些其他属性。例如这里，定义宽为 600. 你可以尝试一下，不写这个特性，看看效果如何。

altair 最让我喜欢的地方，是它和 matploblib 比起来，更加现代化。和网页可以深度结合，还能支持用户交互。

对普通统计图绘制，最大的好处在于对中文的支持。不需要你像 matplotlib 里面添加许多组件儿，直接就能支撑。

In [39]:
alt.Chart(temp_df).mark_point().encode(
    alt.X("total_money:Q", title="总财富"),
    alt.Y("gini:Q",title="基尼系数"),
    alt.Color("saving:N",title="储蓄")
).properties(width=600)

alt.Chart(...)

## 思考题



第 1 题：在散点图的基础上，如果我们希望用折线代表趋势，该怎么办？

In [43]:
#hide

base = alt.Chart(temp_df).mark_point().encode(
    alt.X("total_money:Q", title="总财富"),
    alt.Y("gini:Q",title="基尼系数"),
    alt.Color("saving:N",title="储蓄")
).properties(width=600)
base + base.mark_line()

alt.LayerChart(...)

第 2 题：如果我们不希望用散点图，而是打算用柱状图来描述相同的一组数据，像下图这样，该如何做？

In [42]:
#hide

alt.Chart(temp_df).mark_bar().encode(
alt.X("saving:N", title="储蓄"),
    alt.Y("gini:Q",title="基尼系数"),
    alt.Color("saving:N",title="储蓄"),
    alt.Column("total_money:Q", title="总财富")
)

alt.Chart(...)

第 3 题：刚才的图形里面，咱们只截取了 `num_agents` 为 100 人的情况。如果我们希望用一张图，就把 `num_agents` 的所有取值都展现出来。就像下图一样，该怎么做？

In [38]:
#hide

alt.Chart(df).mark_point().encode(
    alt.X("total_money:Q"),
    alt.Y("gini:Q"),
    alt.Color("saving:N"),
    alt.Row("num_agents:Q")
).properties(width=600)

alt.Chart(...)